In [1]:
# Local imports
import argparse
import glob
import os
import sys
import re

# Third-party imports
import xarray as xr
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import cftime

In [2]:
# Import the dictionaries
import dictionaries as dic

# Import the functions
import processing_NAO_data as pnd
import functions_lag as fnc_lag
import functions_skill_maps as fnc_sm

In [3]:
# First thing to do is to process the observations
# We need to set up the parameters for this
forecast_range = "2-6"
season = "DJFM"
variable = "psl"
obs_var_name = "psl"
region_grid = "north-atlantic"
models = dic.models # all models for psl
observations_path = dic.full_era5 # contains all months and variables

In [4]:
# First use the function to process the data for the azores region
# define the region
region = "azores"

# here we are using the function process_observations
# from functions_skill_maps.py
obs_azores = fnc_sm.process_observations(variable, region, region_grid,
                                          forecast_range, season, 
                                          observations_path, obs_var_name)

Forecast range: 2 - 6
Rolling mean range: 5


In [5]:
# Now do the same but for the iceland region
region = "iceland"

# Run the function process_observations
obs_iceland = fnc_sm.process_observations(variable, region, region_grid,
                                          forecast_range, season, 
                                          observations_path, obs_var_name)

Forecast range: 2 - 6
Rolling mean range: 5


In [6]:
# Now we have obs anomaly fields for year 2-6 DJFM psl
# for both the azores and iceland gridboxes
# we want to take the fieldmean of these
# then subtract iceland from azores
# use the calculate_nao_index function from functions_skill_maps.py for this
obs_nao_index = fnc_sm.calculate_nao_index(obs_azores, obs_iceland)

In [11]:
# obs_nao_index.values

In [10]:
# obs_nao_index.time

In [ ]:
# Now we want to load all of the same init files
# There should be 158 of them (we are missing CanESM5)
# These contain year 2-6 running means
# We will use them for our no lag case